In [ ]:
!pip install PyPDF2
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._dete

In [15]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch

In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
model = "tiiuae/falcon-7b-instruct"

In [3]:
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")

In [4]:
text_split = []
metadata=[]
chunk_size = 1024
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})

In [5]:
embeddings_db = embeddings.encode(text_split)

In [6]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [13]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3116532380.py, line 5)

In [8]:
prompt = "Máquina virtual java"

prompt_embeddings = embeddings.encode([prompt])

results = collection.query(
    query_embeddings=prompt_embeddings,
    n_results=2,
    where={"name": "book-langchain"}
    )

data = results['documents'][0]
distance = results['distances']
print(data)

for i in range(2):
    if distance[0][i] > 0.9:
        data = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
print(distance)
print(data)


['onal. Exatamente! Essa máquina existe e se chama Máquina Virtual Java, ou JVM.\nA Máquina Virtual Java provê especiﬁ  cações de plataforma de \nhardware  na qual se compila todo código de tecnologia Java. Essas especi-\nﬁ cações permitem que o software  Java seja uma plataforma independente, \npois a compilação é feita por uma máquina genérica, a JVM.\nO resultado de uma compilação de um código fonte Java é o \nbytecode , que é uma linguagem de máquina inteligível para a JVM. \nO bytecode  é independente de hardware ; assim, basta o computador ou o \ndispositivo eletrônico (como um celular ou televisão) ter o interpretador adequado (a JVM) que poderá executar um programa Java compilado, não importando em que tipo de computador ele foi compilado.\nGarbage Collection\nVisualize a memória do seu computador como uma grande rua \nonde uma equipe de gandulas de tempos em tempos vai limpando tudo que não está sendo utilizado pelo Java. Isso mesmo. Essa equipe presta ao sistema operacional u

In [9]:
torch.manual_seed(4)

prompt_template = f"""You are a chatbot assistant. 

    Answer in portuguese the question based only on the context below:    
    {data}

"""

sequences = pipe(
    prompt_template,
    max_new_tokens=1024,
    do_sample=False, #introduz um elemento de aleatoriedade, injetando criatividade na produção do LLM
    top_k=10, #número de escolhas de vocabulário de maior probabilidade
    return_full_text = False,
    temperature=0.1
)

for seq in sequences:

    print(f"Result: {seq['generated_text']}")

/home/nayra/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nayra/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:562: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: # Compile the code and get the bytecode
code = '[on. Exatamente! Essa máquina existe e se chama Máquina Virtual Java, ou JVM. A Máquina Virtual Java provê especi- cidades de plataforma de hardware na qual se compila todo código de tecnologia Java. Essas especi- cidades permitem que o software Java seja uma plataforma independente, pois a compilação é feita por uma máquina genérica, a JVM. O resultado de uma compilação de um código fonte Java é o bytecode, que é uma linguagem de máquina inteligível para a JVM. O bytecode  é independente de hardware ; assim, basta o computador ou o dispositivo eletrônico (como um celular ou televisão) ter o interpretador adequado (a JVM) que poderá executar um programa Java compilado, não importando em que tipo de computador ele foi compilado. Garbage Collection Visualize a memory of your computer as a large road where garbage trucks are constantly cleaning up. This is similar to the way Java works, where it cleans up the memory it used to execut